In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

c:\users\professional\appdata\local\programs\python\python37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
c:\users\professional\appdata\local\programs\python\python37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
c:\users\professional\appdata\local\programs\python\python37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
gp_table = pd.read_table('./crab_gp_kaz_10_2016-2018_calib_7types.csv', sep='\t')

In [3]:
dataset_pulse = pd.DataFrame(columns=[
    f'point_{i}' for i in range(100)
])
dataset_pulse['type'] = None

In [5]:
for idx, _ in enumerate(gp_table['Date']):
    file = './results_by_08.05.2019/' + gp_table['fName'][idx][2:]
    profile = np.genfromtxt(file, skip_header=6)
    
    if len(profile) == 100:
        pass
    else:
        profile = np.append(profile, np.zeros(100 - len(profile)))
    
    profile = profile/max(profile)
    
    dataset_pulse.loc[idx] = [
        *profile,
        int(gp_table['Type'][idx])
    ]

In [6]:
dataset_pulse

,point_0,point_1,point_2,point_3,point_4,point_5,point_6,point_7,point_8,point_9,...,point_91,point_92,point_93,point_94,point_95,point_96,point_97,point_98,point_99,type
0,-0.116068,0.056792,-0.039856,-0.049190,-0.099734,0.139417,0.389470,0.649634,0.793392,0.909435,...,0.150278,0.128720,0.113064,0.179966,0.093957,0.176045,0.115952,0.194360,0.199105,6.0
1,-0.004846,-0.002597,0.055012,-0.011008,0.049727,0.262528,0.564671,0.796120,0.806760,0.868158,...,0.079857,0.097565,0.096660,0.107766,0.150606,0.217091,0.186191,0.241283,0.147534,6.0
2,-0.051723,-0.057164,-0.087540,0.081373,-0.020544,-0.064333,0.364793,0.665026,0.714265,0.789584,...,0.144582,0.119296,0.067469,0.090157,0.004430,0.202681,0.038197,0.141786,0.176073,6.0
3,-0.133650,0.001703,0.037257,-0.034801,-0.054465,0.027149,0.071309,0.533312,0.825206,0.974249,...,0.122720,0.123886,0.145409,0.160067,0.139435,0.133456,0.116303,0.099176,0.074070,6.0
4,-0.104604,0.084858,0.109028,0.104102,0.133220,-0.013480,0.104511,0.537790,0.701288,0.895597,...,0.076261,0.081273,0.148621,0.016534,0.127125,0.127706,0.009056,0.144541,0.136080,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2618,0.193209,0.096078,0.105868,0.181870,-0.017440,-0.398477,-0.045887,0.369273,0.263771,0.536943,...,-0.052520,-0.088771,0.237192,-0.202880,-0.213841,0.017357,-0.010464,-0.131797,-0.019804,4.0
2619,0.244981,0.212606,-0.013977,-0.080825,0.164749,0.137054,-0.108618,0.030646,0.152574,0.542721,...,0.160328,0.272256,0.134943,-0.144215,0.028349,0.112397,-0.042104,0.070811,-0.048475,4.0
2620,-0.058014,-0.092051,-0.227012,-0.109336,-0.031386,0.087405,0.430110,0.871916,0.838833,0.757797,...,-0.055858,0.006071,0.009553,0.061458,-0.091841,-0.006158,-0.072883,-0.241286,-0.119026,4.0
2621,-0.161870,-0.016245,-0.039036,-0.137689,-0.111197,0.131454,-0.133019,-0.177296,0.095535,0.950717,...,0.060895,-0.286406,-0.104043,-0.108564,0.180078,0.009290,0.038477,-0.012519,-0.110675,4.0


In [14]:
# reset type to 2 fraction 0 - noise and 1 - pulse
for idx, _ in enumerate(dataset_pulse['point_0']):
    if dataset_pulse['type'][idx] < 4:
        dataset_pulse['type'][idx] = 0
    else:
        dataset_pulse['type'][idx] = 1

In [15]:
X = dataset_pulse.drop(['type'], axis=1)
y = dataset_pulse['type']

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [29]:
clf = DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print(accuracy_score(predictions, y_test))

0.9148665819567979


In [30]:
clf = KNeighborsClassifier()
clf = clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print(accuracy_score(predictions, y_test))

0.9339263024142312


In [31]:
clf = RandomForestClassifier()
clf = clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print(accuracy_score(predictions, y_test))

0.9491740787801779


c:\users\professional\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [32]:
clf = LogisticRegression()
clf = clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print(accuracy_score(predictions, y_test))

0.9199491740787802


c:\users\professional\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [33]:
clf = GaussianNB()
clf = clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print(accuracy_score(predictions, y_test))

0.9021601016518425
